# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [14]:
# Dependencies and Setup
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
import hvplot.pandas
from math import radians, cos, sin, asin, sqrt
from bokeh.resources import INLINE

# Import API key
from api_keys import geoapify_key
from api_keys import weather_api_key

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../Weatherpy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kerrville,30.0474,-99.1403,23.03,25,0,5.66,US,1674070016
1,1,port lincoln,-34.7333,135.8667,15.46,59,100,7.38,AU,1674070017
2,2,norman wells,65.2820,-126.8329,-19.98,77,100,1.54,CA,1674070018
3,3,solnechnyy,50.7214,136.6319,-28.04,98,100,0.66,RU,1674070019
4,4,barrow,71.2906,-156.7887,-19.99,77,100,8.75,US,1674070013


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [21]:
%%capture --no-display

# Configure the map plot
map_cities = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color = "City",
    xlabel='Latitude', ylabel='Longitude'
)
# save the map
hvplot.save(map_cities, "output_data/map_cities.html", resources=INLINE)

# Display the map plot
map_cities



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# This module calculates the distance in km between two points on the globe and is used further in this notebook to calculate distances.

def distance(lat1, lat2, lon1, lon2):
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))
    r = 6371
    return(c * r)
     
    

In [24]:
# Narrow down cities that fit criteria and drop any results with null values
index_reg = len(city_data_df)
ideal_cities=[]
nogo_cities=[]
# if query is run more than once we want to delete the last query resulst
city_data_df=city_data_df.dropna(subset=['City_ID'])

current_city = input("Which city/town/village do you live? ")
current_state = input("In which state is that? ")
current_country = input("In which country? ")
ideal_temp_min = input("What is the desired minimum temerature (C)? ")
ideal_temp_max = input("What is the desired maximum temerature (C)? ")
travel_distance = input("How far do you want to travel in km as the crow flies?)? ")


# query geoapify for coordinates current location
geo_query_input = f"{current_city},{current_state},{current_country}"
params = {
    "apiKey":geoapify_key,
    "format":"json",
    "text":geo_query_input
            }
base_geourl = "https://api.geoapify.com/v1/geocode/search"
response = requests.get(base_geourl, params=params).json()
current_lat = response["results"][0]["lat"]
current_lon = response["results"][0]["lon"]
city_id = response["results"][0]["place_id"]
country_id = response["results"][0]["country"]

# query weather for weather data as per geo coordinates
units ="metric"
url_weatherhome = f"https://api.openweathermap.org/data/2.5/weather?lat={current_lat}&units={units}&lon={current_lon}&appid={weather_api_key}"
response_weatherhome = requests.get(url_weatherhome).json()

# add the results to the city_data_df
home_max = response_weatherhome['main']['temp_max']
home_humid = response_weatherhome['main']['humidity']
home_cloud = response_weatherhome['clouds']['all']
home_wind = response_weatherhome["wind"]['speed']
home_country = response_weatherhome['sys']['country']
home_date = response_weatherhome["dt"]

# set correct dtypes
travel_distance=float(travel_distance)
current_lat=float(current_lat)
current_lon=float(current_lon)
ideal_temp_max=float(ideal_temp_max)
ideal_temp_min=float(ideal_temp_min)

# add current city to city_data_df
city_data_df.loc[index_reg, "Lat"] = current_lat
city_data_df.loc[index_reg, "Lng"] = current_lon
city_data_df.loc[index_reg, "City"] = current_city
city_data_df.loc[index_reg, "Max Temp"] = home_max
city_data_df.loc[index_reg, "Humidity"] = home_humid
city_data_df.loc[index_reg, "Cloudiness"] = home_cloud
city_data_df.loc[index_reg, "Wind Speed"] = home_wind
city_data_df.loc[index_reg, "Country"] = home_country
city_data_df.loc[index_reg, "Date"] = home_date


for index, row in city_data_df.iterrows():
        city_ideal = row["City"]
        city_lat = row["Lat"]
        city_lng = row["Lng"]
        temp = row["Max Temp"]
        distance_fromhome= distance(current_lat, city_lat, current_lon,city_lng)
        if distance_fromhome < travel_distance:
                if temp > ideal_temp_min:
                        if temp < ideal_temp_max: 
                                ideal_cities.append(city_ideal)
                                # print (f"{city_ideal} is only {distance_fromhome} km away" )
                        

        else:
                nogo_cities.append(city_ideal)
                
ideal_cities = pd.DataFrame(ideal_cities, columns =["City"])
ideal_cities = pd.merge(ideal_cities, city_data_df, on="City", how="left",left_index=False)    

### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities.copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df ["Hotel Name"]=""

# Display sample data
hotel_df.sample(5)

,City,City_ID,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
20,kalmunai,192.0,7.4167,81.8167,25.06,64.0,100.0,4.30,LK,1.674070e+09,
11,amahai,113.0,-3.3333,128.9167,25.62,84.0,94.0,2.39,ID,1.674070e+09,
35,tevaitoa,333.0,-16.7833,-151.5000,26.84,74.0,20.0,6.87,PF,1.674070e+09,
1,hithadhoo,15.0,-0.6000,73.0833,27.70,71.0,10.0,4.06,MV,1.674070e+09,
39,samarai,418.0,-10.6167,150.6667,27.00,81.0,100.0,4.73,PG,1.674070e+09,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 5000
categories = "accommodation.hotel"
# limit = 20

    # YOUR CODE HERE
params = {"radius":radius,
            "categories":categories,
            # "limit":limit,
            "apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    srch_lat = row["Lat"]
    srch_lon = row["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary


    params["filter"] = f"circle:{srch_lon},{srch_lat},{radius}"
    params["bias"] = f"proximity:{srch_lon},{srch_lat}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.sample(5)
# print(json.dumps(name_address, indent=4, sort_keys=True))

Starting hotel search
faanui - nearest hotel: Oa Oa Lodge
hithadhoo - nearest hotel: Pebbles Inn
saint-philippe - nearest hotel: Le Baril
georgetown - nearest hotel: Page 63 hostel
alofi - nearest hotel: No hotel found
rikitea - nearest hotel: Chez Bianca & Benoit
le port - nearest hotel: No hotel found
vaini - nearest hotel: Keleti International Resort
atuona - nearest hotel: Pearl Resort
butaritari - nearest hotel: No hotel found
east london - nearest hotel: No hotel found
amahai - nearest hotel: No hotel found
mahebourg - nearest hotel: Grand Bel Air
teahupoo - nearest hotel: Vanira Lodge
lorengau - nearest hotel: Seeadler Bay Hotel
siocon - nearest hotel: No hotel found
hilo - nearest hotel: Dolphin Bay Hotel
kavaratti - nearest hotel: No hotel found
santa rosa - nearest hotel: JARS RESORT
kloulklubed - nearest hotel: The Adventures Inn
kalmunai - nearest hotel: Saji new food
kirakira - nearest hotel: No hotel found
paciran - nearest hotel: No hotel found
dhidhdhoo - nearest hotel:

,City,City_ID,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
28,waingapu,264.0,-9.6567,120.2641,25.42,84.0,100.0,2.13,ID,1.674070e+09,HOTEL SANDLEWOOD
18,santa rosa,165.0,14.3122,121.1114,25.99,85.0,68.0,6.32,PH,1.674070e+09,JARS RESORT
39,samarai,418.0,-10.6167,150.6667,27.00,81.0,100.0,4.73,PG,1.674070e+09,No hotel found
2,saint-philippe,22.0,-21.3585,55.7679,26.77,91.0,100.0,6.82,RE,1.674070e+09,Le Baril
43,coihaique,475.0,-45.5752,-72.0662,29.97,21.0,0.0,4.12,CL,1.674071e+09,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

# Configure the map plot

# dont display if no hotel found
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] !="No hotel found"]


map_vacation= hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color = "City",
    xlabel='Latitude', ylabel='Longitude',
    hover_cols=["City","Country","Hotel Name","Lat","Lng","Humidity"])
    # hover_cols=["City","Country","Hotel Name","Lat","Lng","Humidity"])

# save the map
hvplot.save(map_vacation, "output_data/map_vacation.html", resources=INLINE)

# Display the map
map_vacation


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)